In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('/content/drive/MyDrive/NeuralNetworks/Sentiment.csv')

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

# Preprocess the text data
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
data['text'] = data['text'].apply(lambda x: x.replace('rt', ' '))  # Remove 'rt' (Retweets)

<ipython-input-3-4a1f1d64d4bf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-3-4a1f1d64d4bf>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
<ipython-input-3-4a1f1d64d4bf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [4]:
# Define the function to create the LSTM model
def createmodel():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Label Encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [6]:
# LSTM Model Architecture
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Summary
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
291/291 - 65s - loss: 0.8270 - accuracy: 0.6434 - val_loss: 0.7550 - val_accuracy: 0.6785 - 65s/epoch - 222ms/step
Epoch 2/10
291/291 - 43s - loss: 0.6763 - accuracy: 0.7080 - val_loss: 0.7555 - val_accuracy: 0.6765 - 43s/epoch - 149ms/step
Epoch 3/10
291/291 - 40s - loss: 0.6189 - accuracy: 0.739

In [7]:
# Evaluate the model on test data
score, accuracy = model.evaluate(X_test, y_test, verbose=2, batch_size=32)
print("Test Loss:", score)
print("Test Accuracy:", accuracy)

144/144 - 1s - loss: 1.2004 - accuracy: 0.6566 - 1s/epoch - 9ms/step
Test Loss: 1.2003941535949707
Test Accuracy: 0.656618595123291


In [8]:
# Save the trained model
model.save('sentimentAnalysis.h5')

In [9]:
from keras.models import load_model

model = load_model('sentimentAnalysis.h5')

In [10]:
# Define the text data to predict sentiment
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing. @realDonaldTrump']

# Tokenize and pad the sentence
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)

In [11]:
# Make predictions using the loaded model
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0]

# Convert sentiment probabilities to sentiment label
sentiment = np.argmax(sentiment_probs)

# Print the sentiment label
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 0s - 371ms/epoch - 371ms/step
Positive


In [12]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

In [13]:

# Now you can proceed with the GridSearchCV
model = KerasClassifier(build_fn=createmodel, verbose=2)
batch_size = [10, 20, 40]
epochs = [1, 2]
param_grid = {'batch_size': batch_size, 'epochs': epochs}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)


# Print the best score and best hyperparameters found by GridSearchCV
print("Best Score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<ipython-input-13-487242306e07>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel, verbose=2)


744/744 - 112s - loss: 0.8278 - accuracy: 0.6445 - 112s/epoch - 150ms/step
186/186 - 3s - loss: 0.7598 - accuracy: 0.6719 - 3s/epoch - 18ms/step


744/744 - 110s - loss: 0.8188 - accuracy: 0.6520 - 110s/epoch - 148ms/step
186/186 - 2s - loss: 0.7560 - accuracy: 0.6729 - 2s/epoch - 13ms/step


744/744 - 110s - loss: 0.8271 - accuracy: 0.6408 - 110s/epoch - 148ms/step
186/186 - 2s - loss: 0.7507 - accuracy: 0.6837 - 2s/epoch - 11ms/step


744/744 - 109s - loss: 0.8228 - accuracy: 0.6500 - 109s/epoch - 147ms/step
186/186 - 3s - loss: 0.7634 - accuracy: 0.6631 - 3s/epoch - 16ms/step


...

Epoch 1/2
186/186 - 36s - loss: 0.8493 - accuracy: 0.6363 - 36s/epoch - 196ms/step
Epoch 2/2
186/186 - 27s - loss: 0.6908 - accuracy: 0.7078 - 27s/epoch - 145ms/step
47/47 - 1s - loss: 0.7395 - accuracy: 0.6778 - 739ms/epoch - 16ms/step


Epoch 1/2
186/186 - 37s - loss: 0.8498 - accuracy: 0.6297 - 37s/epoch - 198ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6849 - accuracy: 0.7061 - 26s/epoch - 141ms/step
47/47 - 1s - loss: 0.7566 - accuracy: 0.6810 - 724ms/epoch - 15ms/step


Epoch 1/2
186/186 - 38s - loss: 0.8472 - accuracy: 0.6334 - 38s/epoch - 206ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6830 - accuracy: 0.7081 - 24s/epoch - 131ms/step
47/47 - 1s - loss: 0.7539 - accuracy: 0.6825 - 1s/epoch - 22ms/step


Epoch 1/2
186/186 - 37s - loss: 0.8542 - accuracy: 0.6328 - 37s/epoch - 199ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6804 - accuracy: 0.7091 - 25s/epoch - 134ms/step
47/47 - 1s - loss: 0.7755 - accuracy: 0.6642 - 746ms/epoch - 16ms/step


Epoch 1/2
233/233 - 43s - loss: 0.8382 - accuracy: 0.6407 - 43s/epoch - 184ms/step
Epoch 2/2
233/233 - 33s - loss: 0.6868 - accuracy: 0.7063 - 33s/epoch - 143ms/step
Best Score: 0.675777 using {'batch_size': 40, 'epochs': 2}
